In [307]:
import requests
import json
import pandas as pd
import time

# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

# Running with gunicorn (heroku, digitalocean, etc):
# Procfile >
# web: gunicorn wsgi --log-file -
# wsgi.py >
# from api import app as application
#
#if __name__ == "__main__":
#    application.run()

# https://devcenter.heroku.com/articles/getting-started-with-python
# https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-gunicorn-and-nginx-on-ubuntu-14-04

In [ ]:
# Set up heroku database > 
# heroku addons:create heroku-postgresql:hobby-dev
# Postgres db now up
# Look at config vars (should have DATABASE_URL) >
# heroku config
# Connect to remote psql > 
# heroku pg:psql
#


In [311]:

baseurl = 'https://vast-depths-89393.herokuapp.com/'
# baseurl = 'http://127.0.0.1:5000/'

In [312]:
url = baseurl +'createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [313]:
response.json()

{'signingKey_hex': 'ee3dd7528e7ddfabfb404cc4888c3360a8918a755b08ca9699224794d626d033',
 'traderId': '3',
 'verifyKey_hex': '0c132fc17e4fe7980d471f0d3792bd18330fefde961104a43c93c39ef95af936'}

In [314]:
url = baseurl + 'createMarket'

mkId = 1
# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

In [315]:
content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": mkId, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [316]:
pd.DataFrame(response.json(), index=[0])


,checks,marketBranchId,marketMax,marketMin,marketRootId,traderId
0,True,1,1,0,1,3


In [317]:
# Create a trade
url = baseurl+'createTrade'

content_maketrade = {"signingKey_hex": sk, "traderId": tId, "verifyKey_hex": vk,
  "marketRootId": mkId, "marketBranchId": 1, "price": 0.551, "quantity":1}

In [318]:
# Post trade
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [319]:
# View json response
pd.DataFrame(response.json(), index=[0])

,checks,marketBranchId,marketRootId,price,quantity,traderId
0,"(True, True)",1,1,0.551,1,3


In [ ]:
# Create a matched trade
url = baseurl+'createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [ ]:
response.json()

In [ ]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


In [ ]:
url = baseurl+'createMarket'

for i in range(4):
    t = time.time()

    content_makemarket = {"signingKey_hex": sk,
      "traderId": int(tId), "verifyKey_hex": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

In [ ]:
response.json()

In [ ]:
url = baseurl+'viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)



In [ ]:
a = pd.read_json(response.json())
a

In [ ]:
url = baseurl+'viewOrderBook'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [ ]:
a = pd.read_json(response.json())
a

In [ ]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = 1
    content_maketrade['price'] = 0.44
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


In [ ]:
url = baseurl+'viewOpenTrades'

content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [ ]:
pd.read_json(response.json())

In [ ]:
url = baseurl+'viewMatchedTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [ ]:
pd.read_json(response.json())

In [ ]:
tId